# Web Scraping: Website Scrap

## Iswarya SIVASUBRAMANIAM DIA1

In [1]:
import bs4
from urllib import request
import pandas as pd

import requests
import io
import pdfplumber
import re

In [2]:
#URL du site à scraper
url_ligue_1="https://www.fiches-auto.fr/articles-auto/voiture-electrique/s-852-comparatif-des-voitures-electriques.php"

In [3]:
request_text = request.urlopen(url_ligue_1).read()

In [4]:
htmlpage = bs4.BeautifulSoup(request_text, "html")

In [5]:
# création d'une  liste pour stocker les élements du site
td_elements=[]

In [6]:
tables = htmlpage.find_all('table')

In [7]:
# Nous scrapons les éléments de chaque cellules des tables du site internet
#Les 3 premières tables de notre site ne continennet pas les informations interessantes:
for index, table in enumerate(tables[:-3]):

    print(f"Table {index + 1}:") 
    rows = table.find_all('tr')
    is_first_row=True
    for row in rows:
        if is_first_row:
            is_first_row = False
            continue 
        columns = row.find_all(['th', 'td'])
        for col in columns:
            td_elements.append(col.text.strip())
            print(col.text.strip(), end='\t')
        print() 
    print()  

Table 1:
Ami (8 ch)	0Sec.	45km/h	0Dm3	5.5kWh	70km	

Table 2:
C-Zero (67 ch)	15.9Sec.	130km/h	166Dm3	14.5kWh	100km	
Spring (45 ch)	19.1Sec.	125km/h	270Dm3	26.8kWh	230km	
Spring (65 ch)	13.7Sec.	125km/h	270Dm3	26.8kWh	220km	
500e (95 ch)	9.5Sec.	135km/h	185Dm3	21.3kWh	190km	
500e (118 ch)	9Sec.	150km/h	185Dm3	37.3kWh	321km	
T03 (109 ch)	12Sec.	130km/h	210Dm3	41.3kWh	417km	
Leaf (109 ch)	11.5Sec.	144km/h	370Dm3	22kWh	199km (NEDC)	
Ion (67 ch)	15.9Sec.	130km/h	166Dm3	14.5kWh	100km	
	0 à 100	V max	Coffre	Bat.kWh	Autonomie	
Twingo III (82 ch)	12.9Sec.	135km/h	240Dm3	22kWh	190km	
Zoe (88 ch)	13.5Sec.	135km/h	338Dm3	22kWh	210km (NEDC)	
Zoe (77 ch)	15.5Sec.	135km/h	338Dm3	41kWh	400km (NEDC)	
Zoe (88 ch)	13.2Sec.	135km/h	338Dm3	41kWh	300km	
Zoe (92 ch)	13.2Sec.	135km/h	338Dm3	41kWh	317km	
Zoe (108 ch)	11.4Sec.	135km/h	338Dm3	41kWh	300km	
Mii EV (83 ch)	12.3Sec.	130km/h	251Dm3	32.3kWh	258km	
Fortwo (82 ch)	11.5Sec.	130km/h	260Dm3	17.6kWh	135km	
E-Up! (82 ch)	12.4Sec.	130km/h	250Dm3	18.7kWh	160km 

In [8]:
# cération d'un DataFrame pour stocker les élements du site
Modèle_voiture = []
Acceleration = []
Vmax = []
Coffre = []
Batterie = []
Autonomie = []
for i in range(0, len(td_elements), 6):  
    car_name = td_elements[i]
    Acceleration_value = td_elements[i + 1]
    Vmax_value = td_elements[i + 2]
    Coffre_value = td_elements[i + 3]
    Batterie_value = td_elements[i + 4]
    Autonomie_value = td_elements[i + 5]

    Modèle_voiture.append(car_name)
    Acceleration.append(Acceleration_value)
    Vmax.append(Vmax_value)
    Coffre.append(Coffre_value)
    Batterie.append(Batterie_value)
    Autonomie.append(Autonomie_value)

data = {
    'Car Name': Modèle_voiture,
    'Acceleration 0 à 100(Sec)': Acceleration,
    'Max Speed (km/h)': Vmax,
    'Cargo Volume (Dm3)': Coffre,
    'Battery (kWh)': Batterie,
    'Autonomie (km)': Autonomie,
}
df = pd.DataFrame(data)

In [9]:
df

,Car Name,Acceleration 0 à 100(Sec),Max Speed (km/h),Cargo Volume (Dm3),Battery (kWh),Autonomie (km)
0,Ami (8 ch),0Sec.,45km/h,0Dm3,5.5kWh,70km
1,C-Zero (67 ch),15.9Sec.,130km/h,166Dm3,14.5kWh,100km
2,Spring (45 ch),19.1Sec.,125km/h,270Dm3,26.8kWh,230km
3,Spring (65 ch),13.7Sec.,125km/h,270Dm3,26.8kWh,220km
4,500e (95 ch),9.5Sec.,135km/h,185Dm3,21.3kWh,190km
...,...,...,...,...,...,...
218,EQS (523 ch),4.3Sec.,210km/h,610Dm3,107.8kWh,651km
219,Taycan (625 ch),3.2Sec.,260km/h,366Dm3,83.7kWh,450km
220,Taycan (625 ch),2.8Sec.,260km/h,366Dm3,83.7kWh,412km
221,Taycan Cross Turismo (625 ch),3.3Sec.,250km/h,405Dm3,83.7kWh,452km


In [10]:
df.to_csv('fiche_auto.csv', index=False)

## Scraping from a PDF

In [11]:
import tabula as tb
import pandas as pd
import re

file = 'comparatif_VE.pdf'
data1 = tb.read_pdf(file, area = (32, 0,1000, 100000), pages = '1')
data2=tb.read_pdf(file, area = (0, 0,700, 100000), pages = '2')
data3=tb.read_pdf(file, area = (0, 0,100, 100000), pages = '3')

In [12]:
column_name=['Marque / Modèle',	'Capacité Batterie (brute / utile)',	'Puissance/couple moteur(s) ch / kW / Nm',	'Temps 0-100km/h',	'Vitesse maximale',	'Poids à Vide/Cx/Scx',	'Autonomie/consommation constructeur (cycle WLTP)',	'Estimation autonomie / conso. réelle en ville (à 23°C et -10°C)',	'Estimation autonomie / conso. réelle autoroute (à 23°C et -10°)',	'Modes de charge / puissances',	'Coût recharge 100% de la batterie',	'Coût au 100km (WLTP)',	'Prix du véhicule (TTC - Bonus écologique 6k€ ou 2k€ Inclus)']

In [13]:
data3[0].columns = column_name
data1[0].columns =column_name
data2[0].columns=column_name

In [14]:
data3[0]

,Marque / Modèle,Capacité Batterie (brute / utile),Puissance/couple moteur(s) ch / kW / Nm,Temps 0-100km/h,Vitesse maximale,Poids à Vide/Cx/Scx,Autonomie/consommation constructeur (cycle WLTP),Estimation autonomie / conso. réelle en ville (à 23°C et -10°C),Estimation autonomie / conso. réelle autoroute (à 23°C et -10°),Modes de charge / puissances,Coût recharge 100% de la batterie,Coût au 100km (WLTP),Prix du véhicule (TTC - Bonus écologique 6k€ ou 2k€ Inclus)
0,Tesla Cybertruck Dual Motor (AWD) (2022),120 kWh / 120 kWh,? ch / ? kW / ? Nm,"4,5 sec.",193 km/h,2750kg/ ? / ?,460 km / 261 Wh/km,660-480 km / 182-250 Wh/km,405-325 km / 293-369 Wh/km,Type 2 - 11kW AC / CCS 250kW DC,"17,7€ @ 11kW / 28,8€ (SUC)","3,85€ / 6,26€ (SUC)",53000 € ?
1,Tesla Cybertruck Tri Motor (AWD) (2022),200 kWh / 200 kWh,816 ch / 600 kW / 1400 Nm,3 sec.,209 km/h,3000kg/ ? / ?,750 km / 267 Wh/km,1080-780 km / 185-256 Wh/km,670-540 km / 299-370 Wh/km,Type 2 - 11kW AC / CCS 250kW DC,"29,5€ @ 11kW / 48€ (SUC)","3,96€ / 6,4€ (SUC)",78000 € ?


In [15]:
liste_concat = [data1[0],data2[0],data3[0]]
df_pdf=pd.concat(liste_concat).dropna()
df_pdf= df_pdf.reset_index()
df_pdf = df_pdf.drop('index', axis=1)



In [16]:
df_pdf

,Marque / Modèle,Capacité Batterie (brute / utile),Puissance/couple moteur(s) ch / kW / Nm,Temps 0-100km/h,Vitesse maximale,Poids à Vide/Cx/Scx,Autonomie/consommation constructeur (cycle WLTP),Estimation autonomie / conso. réelle en ville (à 23°C et -10°C),Estimation autonomie / conso. réelle autoroute (à 23°C et -10°),Modes de charge / puissances,Coût recharge 100% de la batterie,Coût au 100km (WLTP),Prix du véhicule (TTC - Bonus écologique 6k€ ou 2k€ Inclus)
0,Citroën Ami (2020) (FWD) >14 ans (2020),"5,8kWh / 5,35kWh",12 ch / 9 kW / 40 Nm,N/D,45 km/h,471 kg / ? / ?,"70-75 km / 78,6 Wh/km",N/D,N/D,"1,84kW AC (Prise domestique)","0,82€ @ 1,8kW","1,09 €",6 000 €
1,Lightyear One (2022) (AWD),60kWh / 60kWh,136 ch / 100 kW / 1200 Nm,10 sec.,150 km/h,"1300 kg /0,2/0,46",725 km / 83 Wh/km,875-505 km / 69-119 Wh/km,565-405 km / 106-148 Wh/km,Type 2 - 22kW AC / CCS 75 kW DC,"8,85€ @ 22kW","1,18 €",149 000 €
2,Renault Twingo Electric (RWD) (2020),"23 kWh / 21,3 kWh",82 ch / 60 kW / 160 Nm,"12,6 sec.",135 km/h,"1208kg/0,33/ 0,72","190 km / 112,1 Wh/km",205-135 km / 104-158 Wh/km,120-90 km / 178-237 Wh/km,Type 2 Accélérée - 22kW AC,"3,14€ @ 22kW","1,65 €",15 350 - 20 450 €
3,Tesla Model 3 - Std range+ (NCA) (RWD)(2019),54kWh / 51kWh,325 ch / 239 kW / 420 Nm,"5,6 sec.",225 km/h,"1700kg/0,23/0,52","448 km / 113,8 Wh/km",515-330 km / 99-155 Wh/km,330-250 km / 155-204 Wh/km,Type 2 - 11kW AC / CCS 170kW DC,"7,52€@11kW / 12,24€ (SUC)","1,67€/2,73€ (SUC)",43 800 €
4,Fiat 500e Hatchback 42 kWh (FWD) (2020),"42 kWh / 37,3 kWh",118 ch / 87 kW / 220 Nm,9 sec.,150 km/h,"1465kg/0,31/0,65","320 km / 116,5 Wh/km",375-240 km / 99-155 Wh/km,225-175 km / 175-230 Wh/km,Type 2 - 11kW AC / CCS 85kW DC,"6,2€ @ 11kW","1,93 €",23 990 - 28900 €
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Byton M-Byte 95 kWh (Chine) (AWD) (2022),105 kWh / 95 kWh,408 ch / 300 kW / 735 Nm,"5,5 sec.",190 km/h,2600kg/ ? / ?,435 km / 218 Wh/km,565-400 km / 168-238 Wh/km,350-280 km / 271-339 Wh/km,Type 2 - 11(22)kW AC / CCS 150kW DC,14€ @ 11kW,"3,22 €",~ 65 000 €
153,Mercedes EQV 300 Long (8 places) (FWD) (2019),100kWh / 90kWh,204 ch / 150 kW / 362 Nm,10 sec.,140 km/h,2635kg/ ? / ?,"363 km / 247,9 Wh/km",480-350 km / 188-257 Wh/km,290-235 km / 310-383 Wh/km,Type 2 - 11kW AC / CCS 110kW DC,"13,3€ @ 11kW","3,66 €",72 744 €
154,Tesla Cybertruck Single Motor (RWD) (2022),100 kWh / 100 kWh,? ch / ? kW / ? Nm,7 sec.,180 km/h,2600kg/ ? / ?,390 km / 256 Wh/km,565-405 km / 177-247 Wh/km,345-275 km / 290-364 Wh/km,Type 2 - 11kW AC / CCS 250kW DC,"14,8€ @ 11kW / 24€ (SUC)","3,78€ / 6,15€ (SUC)",43000 € ?
155,Tesla Cybertruck Dual Motor (AWD) (2022),120 kWh / 120 kWh,? ch / ? kW / ? Nm,"4,5 sec.",193 km/h,2750kg/ ? / ?,460 km / 261 Wh/km,660-480 km / 182-250 Wh/km,405-325 km / 293-369 Wh/km,Type 2 - 11kW AC / CCS 250kW DC,"17,7€ @ 11kW / 28,8€ (SUC)","3,85€ / 6,26€ (SUC)",53000 € ?


In [17]:
df_pdf[['Capacité Batterie brute', 'Capacité Batterie utile']] = df_pdf['Capacité Batterie (brute / utile)'].str.split('/', expand=True)
df_pdf[['Puissance/couple moteur(s) ch', 'Puissance/couple moteur(s) kW','Puissance/couple moteur(s) Nm']] = df_pdf['Puissance/couple moteur(s) ch / kW / Nm'].str.split('/', expand=True)
df_pdf[['Poids à Vide', 'Poids Cx','Poids Scx']] = df_pdf['Poids à Vide/Cx/Scx'].str.split('/', expand=True)
df_pdf[['Autonomie', 'Consommation construsteur (cycle WLTP)']] = df_pdf['Autonomie/consommation constructeur (cycle WLTP)'].str.extract(r'([\d]+ km) / ([\d,]+ Wh/km)')
df_pdf[['Estimation autonomie ville','conso. réelle en ville (à 23°C et -10°C)']]=df_pdf['Estimation autonomie / conso. réelle en ville (à 23°C et -10°C)'].str.extract(r'(\d+-\d+ km) / (\d+-\d+ Wh/km)')
df_pdf[['Estimation autonomie autoroute','conso. réelle autoroute (à 23°C et -10°)']]= df_pdf['Estimation autonomie / conso. réelle autoroute (à 23°C et -10°)'].str.extract(r'(\d+-\d+ km) / (\d+-\d+ Wh/km)')


In [18]:
df_pdf

,Marque / Modèle,Capacité Batterie (brute / utile),Puissance/couple moteur(s) ch / kW / Nm,Temps 0-100km/h,Vitesse maximale,Poids à Vide/Cx/Scx,Autonomie/consommation constructeur (cycle WLTP),Estimation autonomie / conso. réelle en ville (à 23°C et -10°C),Estimation autonomie / conso. réelle autoroute (à 23°C et -10°),Modes de charge / puissances,...,Puissance/couple moteur(s) Nm,Poids à Vide,Poids Cx,Poids Scx,Autonomie,Consommation construsteur (cycle WLTP),Estimation autonomie ville,conso. réelle en ville (à 23°C et -10°C),Estimation autonomie autoroute,conso. réelle autoroute (à 23°C et -10°)
0,Citroën Ami (2020) (FWD) >14 ans (2020),"5,8kWh / 5,35kWh",12 ch / 9 kW / 40 Nm,N/D,45 km/h,471 kg / ? / ?,"70-75 km / 78,6 Wh/km",N/D,N/D,"1,84kW AC (Prise domestique)",...,40 Nm,471 kg,?,?,75 km,"78,6 Wh/km",NaN,NaN,NaN,NaN
1,Lightyear One (2022) (AWD),60kWh / 60kWh,136 ch / 100 kW / 1200 Nm,10 sec.,150 km/h,"1300 kg /0,2/0,46",725 km / 83 Wh/km,875-505 km / 69-119 Wh/km,565-405 km / 106-148 Wh/km,Type 2 - 22kW AC / CCS 75 kW DC,...,1200 Nm,1300 kg,"0,2","0,46",725 km,83 Wh/km,875-505 km,69-119 Wh/km,565-405 km,106-148 Wh/km
2,Renault Twingo Electric (RWD) (2020),"23 kWh / 21,3 kWh",82 ch / 60 kW / 160 Nm,"12,6 sec.",135 km/h,"1208kg/0,33/ 0,72","190 km / 112,1 Wh/km",205-135 km / 104-158 Wh/km,120-90 km / 178-237 Wh/km,Type 2 Accélérée - 22kW AC,...,160 Nm,1208kg,"0,33","0,72",190 km,"112,1 Wh/km",205-135 km,104-158 Wh/km,120-90 km,178-237 Wh/km
3,Tesla Model 3 - Std range+ (NCA) (RWD)(2019),54kWh / 51kWh,325 ch / 239 kW / 420 Nm,"5,6 sec.",225 km/h,"1700kg/0,23/0,52","448 km / 113,8 Wh/km",515-330 km / 99-155 Wh/km,330-250 km / 155-204 Wh/km,Type 2 - 11kW AC / CCS 170kW DC,...,420 Nm,1700kg,"0,23","0,52",448 km,"113,8 Wh/km",515-330 km,99-155 Wh/km,330-250 km,155-204 Wh/km
4,Fiat 500e Hatchback 42 kWh (FWD) (2020),"42 kWh / 37,3 kWh",118 ch / 87 kW / 220 Nm,9 sec.,150 km/h,"1465kg/0,31/0,65","320 km / 116,5 Wh/km",375-240 km / 99-155 Wh/km,225-175 km / 175-230 Wh/km,Type 2 - 11kW AC / CCS 85kW DC,...,220 Nm,1465kg,"0,31","0,65",320 km,"116,5 Wh/km",375-240 km,99-155 Wh/km,225-175 km,175-230 Wh/km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Byton M-Byte 95 kWh (Chine) (AWD) (2022),105 kWh / 95 kWh,408 ch / 300 kW / 735 Nm,"5,5 sec.",190 km/h,2600kg/ ? / ?,435 km / 218 Wh/km,565-400 km / 168-238 Wh/km,350-280 km / 271-339 Wh/km,Type 2 - 11(22)kW AC / CCS 150kW DC,...,735 Nm,2600kg,?,?,435 km,218 Wh/km,565-400 km,168-238 Wh/km,350-280 km,271-339 Wh/km
153,Mercedes EQV 300 Long (8 places) (FWD) (2019),100kWh / 90kWh,204 ch / 150 kW / 362 Nm,10 sec.,140 km/h,2635kg/ ? / ?,"363 km / 247,9 Wh/km",480-350 km / 188-257 Wh/km,290-235 km / 310-383 Wh/km,Type 2 - 11kW AC / CCS 110kW DC,...,362 Nm,2635kg,?,?,363 km,"247,9 Wh/km",480-350 km,188-257 Wh/km,290-235 km,310-383 Wh/km
154,Tesla Cybertruck Single Motor (RWD) (2022),100 kWh / 100 kWh,? ch / ? kW / ? Nm,7 sec.,180 km/h,2600kg/ ? / ?,390 km / 256 Wh/km,565-405 km / 177-247 Wh/km,345-275 km / 290-364 Wh/km,Type 2 - 11kW AC / CCS 250kW DC,...,? Nm,2600kg,?,?,390 km,256 Wh/km,565-405 km,177-247 Wh/km,345-275 km,290-364 Wh/km
155,Tesla Cybertruck Dual Motor (AWD) (2022),120 kWh / 120 kWh,? ch / ? kW / ? Nm,"4,5 sec.",193 km/h,2750kg/ ? / ?,460 km / 261 Wh/km,660-480 km / 182-250 Wh/km,405-325 km / 293-369 Wh/km,Type 2 - 11kW AC / CCS 250kW DC,...,? Nm,2750kg,?,?,460 km,261 Wh/km,660-480 km,182-250 Wh/km,405-325 km,293-369 Wh/km


df_pdf = df_pdf.drop('index', axis=1)


## Merge the tables 

In [28]:
df=pd.read_csv("lunil.csv")

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Modèle                                50 non-null     object 
 1   Longueur (mm)                         50 non-null     int64  
 2   Largeur (mm)                          50 non-null     int64  
 3   Prix (euros) min                      44 non-null     float64
 4   Prix (euros) max                      12 non-null     float64
 5   Accélération de 0 à 100 km/h (s) min  37 non-null     float64
 6   Accélération de 0 à 100 km/h (s) max  37 non-null     float64
 7   Puissance de la batterie (kWh) min    45 non-null     float64
 8   Puissance de la batterie (kWh) max    45 non-null     float64
 9   Vitesse maximale (km/h) min           42 non-null     float64
 10  Vitesse maximale (km/h) max           42 non-null     float64
 11  Autonomie (km) min   

In [30]:
df

,Modèle,Longueur (mm),Largeur (mm),Prix (euros) min,Prix (euros) max,Accélération de 0 à 100 km/h (s) min,Accélération de 0 à 100 km/h (s) max,Puissance de la batterie (kWh) min,Puissance de la batterie (kWh) max,Vitesse maximale (km/h) min,Vitesse maximale (km/h) max,Autonomie (km) min,Autonomie (km) max,Poids (kg) min,Poids (kg) max
0,Volkswagen e-Up!,3595,1641,20000.0,30000.0,9.9,9.9,32.0,32.0,130.0,130.0,260.0,260.0,1100.0,1100.0
1,Fiat 500e,3555,1627,30000.0,NaN,9.1,9.1,24.0,24.0,150.0,150.0,200.0,200.0,1150.0,1150.0
2,Citroën e-Méhari,3495,1660,22000.0,NaN,NaN,NaN,30.0,30.0,110.0,110.0,200.0,200.0,1065.0,1065.0
3,DS3 Crossback E-tense,4118,1791,40000.0,NaN,8.7,8.7,50.0,50.0,150.0,150.0,430.0,430.0,1170.0,1170.0
4,Dacia Spring,3715,1625,20000.0,NaN,19.1,19.1,26.0,36.0,125.0,125.0,155.0,300.0,970.0,970.0
5,Honda e,3945,1755,33500.0,NaN,8.3,8.3,35.5,35.5,145.0,145.0,220.0,220.0,1320.0,1320.0
6,Nissan LEAF II,4480,1820,32000.0,NaN,7.9,7.9,40.0,40.0,150.0,150.0,385.0,385.0,1590.0,1590.0
7,Renault Zoé,4085,1730,35000.0,NaN,9.5,9.5,52.0,52.0,140.0,140.0,395.0,395.0,1502.0,1502.0
8,Opel Corsa-e,4050,1800,29000.0,NaN,7.6,7.6,50.0,50.0,150.0,150.0,330.0,330.0,1365.0,1365.0
9,Peugeot e-208,4055,1745,33000.0,NaN,8.1,8.1,46.0,46.0,150.0,150.0,400.0,400.0,1455.0,1455.0


In [31]:
def extract_second_and_third_word(text):
    words = text.split()
    if len(words) > 1:
        return ' '.join(words[1:3]) if len(words) > 2 else words[1]
    else:
        return None

In [32]:
result_df = pd.DataFrame()

In [33]:
df['SecondWord'] = df['Modèle'].apply(extract_second_and_third_word)

In [34]:
df

,Modèle,Longueur (mm),Largeur (mm),Prix (euros) min,Prix (euros) max,Accélération de 0 à 100 km/h (s) min,Accélération de 0 à 100 km/h (s) max,Puissance de la batterie (kWh) min,Puissance de la batterie (kWh) max,Vitesse maximale (km/h) min,Vitesse maximale (km/h) max,Autonomie (km) min,Autonomie (km) max,Poids (kg) min,Poids (kg) max,SecondWord
0,Volkswagen e-Up!,3595,1641,20000.0,30000.0,9.9,9.9,32.0,32.0,130.0,130.0,260.0,260.0,1100.0,1100.0,e-Up!
1,Fiat 500e,3555,1627,30000.0,NaN,9.1,9.1,24.0,24.0,150.0,150.0,200.0,200.0,1150.0,1150.0,500e
2,Citroën e-Méhari,3495,1660,22000.0,NaN,NaN,NaN,30.0,30.0,110.0,110.0,200.0,200.0,1065.0,1065.0,e-Méhari
3,DS3 Crossback E-tense,4118,1791,40000.0,NaN,8.7,8.7,50.0,50.0,150.0,150.0,430.0,430.0,1170.0,1170.0,Crossback E-tense
4,Dacia Spring,3715,1625,20000.0,NaN,19.1,19.1,26.0,36.0,125.0,125.0,155.0,300.0,970.0,970.0,Spring
5,Honda e,3945,1755,33500.0,NaN,8.3,8.3,35.5,35.5,145.0,145.0,220.0,220.0,1320.0,1320.0,e
6,Nissan LEAF II,4480,1820,32000.0,NaN,7.9,7.9,40.0,40.0,150.0,150.0,385.0,385.0,1590.0,1590.0,LEAF II
7,Renault Zoé,4085,1730,35000.0,NaN,9.5,9.5,52.0,52.0,140.0,140.0,395.0,395.0,1502.0,1502.0,Zoé
8,Opel Corsa-e,4050,1800,29000.0,NaN,7.6,7.6,50.0,50.0,150.0,150.0,330.0,330.0,1365.0,1365.0,Corsa-e
9,Peugeot e-208,4055,1745,33000.0,NaN,8.1,8.1,46.0,46.0,150.0,150.0,400.0,400.0,1455.0,1455.0,e-208


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Modèle                                50 non-null     object 
 1   Longueur (mm)                         50 non-null     int64  
 2   Largeur (mm)                          50 non-null     int64  
 3   Prix (euros) min                      44 non-null     float64
 4   Prix (euros) max                      12 non-null     float64
 5   Accélération de 0 à 100 km/h (s) min  37 non-null     float64
 6   Accélération de 0 à 100 km/h (s) max  37 non-null     float64
 7   Puissance de la batterie (kWh) min    45 non-null     float64
 8   Puissance de la batterie (kWh) max    45 non-null     float64
 9   Vitesse maximale (km/h) min           42 non-null     float64
 10  Vitesse maximale (km/h) max           42 non-null     float64
 11  Autonomie (km) min   

In [35]:
result_df = pd.DataFrame()

In [41]:
df_pdf["Marque / Modèle"] = df_pdf["Marque / Modèle"].str.lower()
df["SecondWord"] = df["SecondWord"].str.lower()

# Effectuer la jointure en utilisant str.contains
merged_df = pd.merge(df_pdf, df, how='left', left_on=lambda x: df_pdf["Marque / Modèle"].str.contains(x).any(), right_on='SecondWord')

KeyError: <function <lambda> at 0x0000027E361513A0>

In [39]:
df_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 27 columns):
 #   Column                                                           Non-Null Count  Dtype 
---  ------                                                           --------------  ----- 
 0   Marque / Modèle                                                  157 non-null    object
 1   Capacité Batterie (brute / utile)                                157 non-null    object
 2   Puissance/couple moteur(s) ch / kW / Nm                          157 non-null    object
 3   Temps 0-100km/h                                                  157 non-null    object
 4   Vitesse maximale                                                 157 non-null    object
 5   Poids à Vide/Cx/Scx                                              157 non-null    object
 6   Autonomie/consommation constructeur (cycle WLTP)                 157 non-null    object
 7   Estimation autonomie / conso. réelle en ville (à 23°C

In [ ]:
for index, row in df1.iterrows():
    second_and_third_words = extract_second_and_third_word(row['Column1'])
    
    # Filter df2 based on whether the second and/or third words appear in the specified column
    filtered_df2 = df2[df2['SomeColumn'].str.contains(second_and_third_words, na=False)]  # Replace 'SomeColumn' with the actual column name
    
    # If there is a match, create a new DataFrame by combining all rows from df1 with all columns from df2 (except 'modèle')
    if not filtered_df2.empty:
        merged_row = pd.concat([row.to_frame().T, filtered_df2.drop(columns=['modèle'])], axis=1)
        
        # Append the result to the final DataFrame
        result_df = result_df.append(merged_row, ignore_index=True)